In [1]:
!nvidia-smi

Sat Dec  3 17:02:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.02    Driver Version: 526.98       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   56C    P8    N/A /  N/A |      0MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Import libraries

In [2]:
import os 
import random 
import pandas as pd 
import torch 

from tqdm import tqdm

In [3]:
def load_feat(path):
    feat = torch.load(path)
    return feat 

In [4]:
def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    
    else:
        return x 
    
    return torch.cat((left, right), dim=0)

In [5]:
def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd 

    if concat_n < 2:
        return x 

    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n)
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2)
    mid = (concat_n // 2)

    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

In [6]:
def preprocess_data(split, feat_dir, phone_path, concat_nframes, \
    train_ratio=0.8, train_val_seed=1337):
    class_num = 41 # pre-computed should not need change 
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}

    if mode != 'test':
        phone_file = open(os.path.join(phone_path, \
            f'{mode}_labels.txt')).readlines() 
        
        for line in phone_file:
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'val':
        # split training and validation data 
        usage_list = open(os.path.join(phone_path, \
            'train_split.txt')).readlines() 
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' \
            else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).\
            readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: \
            PhoneDataset!')
    
    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset - # phone classes: ' + str(class_num) + \
        ' , number of utterances for ' + split + ': ' + \
        str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)

    if mode != 'test':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)

        if mode != 'test':
            label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat 
        if mode != 'test':
            y[idx: idx + cur_len] = label 
        
        idx += cur_len 

    X = X[:idx, :]
    
    if mode != 'test':
        y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)

    if mode != 'test':
        print(y.shape)
        return X, y 
    else:
        return X

# Define dataset

In [7]:
from torch.utils.data import Dataset 
from torch.utils.data import DataLoader 

class LibriDataset(Dataset):

    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None 

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx] 
        else:
            return self.data[idx] 

    def __len__(self):
        return len(self.data)

# Define model

In [8]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 

class BasicBlock(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__() 

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(), 
        )

    def forward(self, x):
        x = self.block(x)
        return x

In [9]:
class Classifier(nn.Module):

    def __init__(self, input_dim, output_dim=41, hidden_layers=1, 
        hidden_dim=256):
        super(Classifier, self).__init__() 

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

# Hyper parameters

In [10]:
# data parameters 
concat_nframes = 1 # the number of frames to concat with
train_ratio = 0.8 # the ratio of data used for training 

# training parameters 
seed = 0 # random seed 
batch_size = 512 # batch size 
num_epochs = 5 # the number of training epoch 
learning_rate = 0.0001 # learning rate
model_path = './model.ckpt' # the path where the checkpoint will be saved

# model parameters 
input_dim = 39 * concat_nframes # the input dim of the model
hidden_layers = 1 # the number of hidden layer 
hidden_dim = 256 # the hidden dim

# Prepare dataset and model

In [11]:
import gc 

# preprocess data
train_X, train_y = preprocess_data(split='train', 
    feat_dir='./dataset/feat', 
    phone_path='./dataset', concat_nframes=concat_nframes, 
    train_ratio=train_ratio)

val_X, val_y = preprocess_data(split='val', 
    feat_dir='./dataset/feat', 
    phone_path='./dataset', concat_nframes=concat_nframes, 
    train_ratio=train_ratio)

[Dataset - # phone classes: 41 , number of utterances for train: 3428


3428it [00:01, 2551.22it/s]


[INFO] train set
torch.Size([2116368, 39])
torch.Size([2116368])
[Dataset - # phone classes: 41 , number of utterances for val: 858


858it [00:00, 2397.75it/s]

[INFO] val set
torch.Size([527790, 39])
torch.Size([527790])


# Get dataset

In [12]:
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# Remove raw feature to save memory

In [13]:
del train_X, train_y, val_X, val_y 
gc.collect()

332

# Get dataloader

In [14]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

In [15]:
device = 'cpu'
print(f'Device: {device}')

Device: cpu


In [16]:
import numpy as np 

# fix seed 
def same_seeds(seed):
    torch.manual_seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    np.random.seed(seed) 
    torch.backends.cudnn.benchmark = False 
    torch.backends.cudnn.deterministic = True 

In [17]:
# fix random seed 
same_seeds(seed)

# create model, define a loss function, and optimizer 
model = Classifier(
    input_dim=input_dim,
    hidden_layers=hidden_layers,
    hidden_dim=hidden_dim
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training

In [18]:
best_acc = 0.0 

for epoch in range(num_epochs):
    train_acc = 0.0
    train_loss = 0.0 
    val_acc = 0.0 
    val_loss = 0.0 

    # training 
    model.train() # set the model to training mode 
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch 
        features = features.to(device)
        labels = labels.to(device) 

        optimizer.zero_grad()
        outputs = model(features)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # get the index of the class with the highest probability
        _, train_pred = torch.max(outputs, 1)
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # validation 
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode 
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch 
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features)

                loss = criterion(outputs, labels)

                _, val_pred = torch.max(outputs, 1)
                # get the index of the class with the highest probability
                val_acc += (val_pred.cpu() == labels.cpu()).\
                    sum().item()
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.\
                format(
                    epoch+1,
                    num_epochs,
                    train_acc/len(train_set),
                    train_loss/len(train_loader)
                )) 
    else:
        print('[{:03d}/{:03d} Train Acc: {:3.6f} Loss: {:3.6f}]'.format(
            epoch+1,
            num_epochs,
            train_acc/len(train_set),
            train_loss/len(train_loader)
        ))

# if not validating, save the last epoch 
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

100%|██████████| 1031/1031 [00:05<00:00, 192.93it/s]


[001/005] Train Acc: 0.421926 Loss: 2.086647


100%|██████████| 1031/1031 [00:05<00:00, 199.24it/s]


[002/005] Train Acc: 0.448990 Loss: 1.934497


100%|██████████| 1031/1031 [00:04<00:00, 208.60it/s]


[003/005] Train Acc: 0.455067 Loss: 1.904358


100%|██████████| 1031/1031 [00:04<00:00, 217.33it/s]


[004/005] Train Acc: 0.458434 Loss: 1.887961


100%|██████████| 1031/1031 [00:05<00:00, 190.55it/s]

[005/005] Train Acc: 0.460749 Loss: 1.876356
